In [1]:
TEST_SPLIT = 0.2
DEV_SPLIT_JOKES = 1
PAD_LEN = 50

EMBEDDING_SIZE = 50

LEARNING_RATE=0.001
EPOCHS=20
BATCH_SIZE=32
VAL_SPLIT=0.3
PATIENCE = 5

In [2]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer

import re

2023-06-15 18:10:30.622951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 18:10:32.918334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
from pathlib import Path
data = Path.cwd().parent / 'raw_data'
data.exists()

True

In [15]:
filename_jokes = '../raw_data/reddit_jokes_rated.csv'
df_import_jokes = pd.read_csv(filename_jokes)
filename_reddit = '../raw_data/reddit/reddit_thread_comments.csv'
df_import_raw = pd.read_csv(filename_reddit, index_col=[0])


/tmp/ipykernel_14361/4014073165.py:4: DtypeWarning: Columns (3,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_import_raw = pd.read_csv(filename_reddit, index_col=[0])


In [16]:
def clean_single_sentence(sentence):
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = re.sub(r'[^\w]', ' ', sentence) ## remove special characters
    return sentence

In [17]:
df_import_raw['comment_body'] = df_import_raw['comment_body'].astype(str)
df_import_jokes['thread_joke'] = [clean_single_sentence(joke) for joke in df_import_jokes['thread_joke']]
df_import_raw['comment_body'] = [clean_single_sentence(joke) for joke in df_import_raw['comment_body']]

In [18]:
sr_sentences = pd.concat([df_import_jokes['thread_joke'], df_import_raw['comment_body']], axis = 0)
ls_sentences = [text_to_word_sequence(_) for _ in sr_sentences]


In [19]:

word2vec = Word2Vec(sentences=ls_sentences, vector_size=EMBEDDING_SIZE, min_count=10, window=5)
vocab_size = len(word2vec.wv)
vocab_size

16453

In [21]:
import pickle

with open("word2vec_v3", 'wb') as file:
    pickle.dump(word2vec, file)
    

In [22]:

with open("word2vec_v3", "rb") as file:
    w2v = pickle.load(file)

In [23]:
w2v